# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/priyankadogra/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/priyankadogra/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/priyankadogra/workspace/aimakerspace/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/priyankadogra/workspace/aimakerspace/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/priyankadogra/workspace/aimakerspace/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '71ae46'. Skipping!
Property 'summary' already exists in node '78f50c'. Skipping!
Property 'summary' already exists in node 'cd216f'. Skipping!
Property 'summary' already exists in node '762d82'. Skipping!
Property 'summary' already exists in node '7e202b'. Skipping!
Property 'summary' already exists in node '9b7855'. Skipping!
Property 'summary' already exists in node 'bffd77'. Skipping!
Property 'summary' already exists in node '1392e2'. Skipping!
Property 'summary' already exists in node '887e7b'. Skipping!
Property 'summary' already exists in node '44462a'. Skipping!
Property 'summary' already exists in node '76fc64'. Skipping!
Property 'summary' already exists in node 'b08436'. Skipping!
Property 'summary' already exists in node '2984e4'. Skipping!
Property 'summary' already exists in node '0649c1'. Skipping!
Property 'summary' already exists in node '279428'. Skipping!
Property 'summary' already exists in node '56d4f6'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '887e7b'. Skipping!
Property 'summary_embedding' already exists in node '9b7855'. Skipping!
Property 'summary_embedding' already exists in node '71ae46'. Skipping!
Property 'summary_embedding' already exists in node 'bffd77'. Skipping!
Property 'summary_embedding' already exists in node '78f50c'. Skipping!
Property 'summary_embedding' already exists in node '762d82'. Skipping!
Property 'summary_embedding' already exists in node '44462a'. Skipping!
Property 'summary_embedding' already exists in node '76fc64'. Skipping!
Property 'summary_embedding' already exists in node '2984e4'. Skipping!
Property 'summary_embedding' already exists in node '1392e2'. Skipping!
Property 'summary_embedding' already exists in node 'cd216f'. Skipping!
Property 'summary_embedding' already exists in node '7e202b'. Skipping!
Property 'summary_embedding' already exists in node 'b08436'. Skipping!
Property 'summary_embedding' already exists in node '0649c1'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 711)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 711)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"Tomlinson et al., 2025 what they say about Cha...",[Introduction ChatGPT launched in November 202...,"The paper by Tomlinson et al., 2025, studies c...",single_hop_specifc_query_synthesizer
1,Who is Collis?,[Table 1: ChatGPT daily message counts (millio...,The context mentions Collis in relation to a s...,single_hop_specifc_query_synthesizer
2,Could you explain the significance of SOC2 cod...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,How does ChatGPT help in decision-making for p...,[Conclusion This paper studies the rapid growt...,ChatGPT likely improves worker output by provi...,single_hop_specifc_query_synthesizer
4,Wha is the varation in ChatGPT usage by occpat...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The first segment shows that ChatGPT usage var...,multi_hop_abstract_query_synthesizer
5,Based on the variation in ChatGPT usage by occ...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context indicates that users in highly pai...,multi_hop_abstract_query_synthesizer
6,"wht types of chatbott interactions (asking, do...",[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context indicates that ChatGPT usage varie...,multi_hop_abstract_query_synthesizer
7,How do Brynjolfsson's findings about AI's impa...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,Brynjolfsson's findings highlight that while A...,multi_hop_specific_query_synthesizer
8,collis how does chatgpt use relate to collis a...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context shows that Collis is mentioned twi...,multi_hop_specific_query_synthesizer
9,"Based on the rapid growth of ChatGPT, which ha...",[<1-hop>\n\nConclusion This paper studies the ...,"The context indicates that by July 2025, ChatG...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'ae5b41'. Skipping!
Property 'summary' already exists in node 'cd54a8'. Skipping!
Property 'summary' already exists in node '8fcfed'. Skipping!
Property 'summary' already exists in node '67d5f6'. Skipping!
Property 'summary' already exists in node '1481e5'. Skipping!
Property 'summary' already exists in node 'e19cdb'. Skipping!
Property 'summary' already exists in node '7ff17e'. Skipping!
Property 'summary' already exists in node '555a9b'. Skipping!
Property 'summary' already exists in node '848135'. Skipping!
Property 'summary' already exists in node '713a13'. Skipping!
Property 'summary' already exists in node '8a8aff'. Skipping!
Property 'summary' already exists in node '0a24b2'. Skipping!
Property 'summary' already exists in node 'f88d77'. Skipping!
Property 'summary' already exists in node '4cb90a'. Skipping!
Property 'summary' already exists in node 'b63508'. Skipping!
Property 'summary' already exists in node '1c85e8'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '8fcfed'. Skipping!
Property 'summary_embedding' already exists in node 'ae5b41'. Skipping!
Property 'summary_embedding' already exists in node 'cd54a8'. Skipping!
Property 'summary_embedding' already exists in node '67d5f6'. Skipping!
Property 'summary_embedding' already exists in node 'b63508'. Skipping!
Property 'summary_embedding' already exists in node '1481e5'. Skipping!
Property 'summary_embedding' already exists in node '713a13'. Skipping!
Property 'summary_embedding' already exists in node '0a24b2'. Skipping!
Property 'summary_embedding' already exists in node 'f88d77'. Skipping!
Property 'summary_embedding' already exists in node '4cb90a'. Skipping!
Property 'summary_embedding' already exists in node '555a9b'. Skipping!
Property 'summary_embedding' already exists in node '7ff17e'. Skipping!
Property 'summary_embedding' already exists in node '8a8aff'. Skipping!
Property 'summary_embedding' already exists in node 'e19cdb'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How does Kulveit et al. contribute to our unde...,[Introduction ChatGPT launched in November 202...,Kulveit et al. is referenced in the context of...,single_hop_specifc_query_synthesizer
1,What are the projected trends and implications...,[Table 1: ChatGPT daily message counts (millio...,"According to the provided data, ChatGPT daily ...",single_hop_specifc_query_synthesizer
2,What is the significance of the SOC in relatio...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,What does the paper reveal about ChatGPT usage...,[Conclusion This paper studies the rapid growt...,"The paper indicates that in the US, ChatGPT us...",single_hop_specifc_query_synthesizer
4,How does the rapid growth and adoption of Chat...,[<1-hop>\n\nConclusion This paper studies the ...,"The rapid growth and adoption of ChatGPT, with...",multi_hop_abstract_query_synthesizer
5,Based on the data showing that ChatGPT message...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The data indicates that ChatGPT's usage has gr...,multi_hop_abstract_query_synthesizer
6,"How does the increase in non-work messages, wh...",[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data shows that non-work messages have gro...,multi_hop_abstract_query_synthesizer
7,Hw does chatGPT classify user intents like ask...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context explains that ChatGPT classifies m...,multi_hop_abstract_query_synthesizer
8,"In US, how ChatGPT usage like in 1-hop and 2-h...",[<1-hop>\n\nConclusion This paper studies the ...,"The 1-hop data indicates that by July 2025, a ...",multi_hop_specific_query_synthesizer
9,Considering the rapid growth of ChatGPT usage ...,[<1-hop>\n\nConclusion This paper studies the ...,"The rapid adoption of ChatGPT in the US, with ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in many different ways both at work and outside of work. Key uses include:\n\n- Performing workplace tasks that either augment or automate human labor.\n- Producing writing, software code, spreadsheets, and other digital products.\n- Seeking information and advice, similar to but more flexible than traditional web search engines.\n- Using AI as co-workers that produce output or as co-pilots that give advice and improve human problem-solving productivity.\n- Engaging in activities related to self-expression, such as relationships, personal reflection, games, and role play (though these are smaller proportions of usage).\n- Therapy and companionship are also noted as prevalent use cases for generative AI.\n\nOverall, AI is highly flexible and people use it for asking questions, doing tasks, and expressing themselves.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dopeness_evaluator`:

## LangSmith Evaluation

In [29]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'crazy-station-79' at:
https://smith.langchain.com/o/bb09119d-c491-4093-9100-b156a137456d/datasets/4e78be08-3e6b-4d36-980b-27d1262f0bdd/compare?selectedSessions=897d9676-5a8e-4403-ba52-6982f6cfb310




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,how OpenAI chatgpt use like in 2024 and 2025 a...,"Based on the provided context, here are the an...",None,"Based on the data from 2024 and 2025, OpenAI's...",1,1,0,20.813178,00f2624c-17b9-4d64-841c-0a9476f0aad8,8df9ec7d-9b32-46cf-9a20-0aa0745d8d21
1,how OpenAI ChatGPT use grow fast and how much ...,ChatGPT grew rapidly after its launch in Novem...,None,The context shows that OpenAI's ChatGPT launch...,1,1,0,4.913990,2674dd99-bcea-4f96-b09d-b7e1f9d7adcd,fbde0d4b-17d9-4e8f-99e3-1f1f7f3ac02f
2,Considering the rapid growth of ChatGPT usage ...,"Based on the provided context, the widespread ...",None,"The rapid adoption of ChatGPT in the US, with ...",1,1,0,5.641187,c9af4ec1-e7b2-4a84-8d23-a47c8992ca53,4af8381e-4b46-43c1-8fec-4a9e530dde65
3,"In US, how ChatGPT usage like in 1-hop and 2-h...","Based on the provided context, in the US, the ...",None,"The 1-hop data indicates that by July 2025, a ...",1,1,0,5.328451,fc5a3670-fae5-4b82-b258-1968736c2a54,2ec6dbd2-6096-4876-8c8f-bd3ebba8a913
4,Hw does chatGPT classify user intents like ask...,ChatGPT classifies user intents into three cat...,None,The context explains that ChatGPT classifies m...,1,0,0,4.211241,f2998fc9-b545-4e40-a913-e34bfdd2998d,bfca2587-cfe7-4f4b-ad73-eb8508ee45f8
5,"How does the increase in non-work messages, wh...",The increase in non-work messages to over 70% ...,None,The data shows that non-work messages have gro...,1,1,0,4.126211,d852de5b-5dd2-4987-9aec-3ff94aaa3235,4e605c7c-4b2a-454f-9689-619759fa06d8
6,Based on the data showing that ChatGPT message...,The data indicates that while much prior econo...,None,The data indicates that ChatGPT's usage has gr...,1,1,0,5.742911,1b052c4b-f40b-4eb1-baa7-c474d0aa33ce,83d14681-b923-44dd-ae05-8834c95d574c
7,How does the rapid growth and adoption of Chat...,The rapid growth and adoption of ChatGPT—reach...,None,"The rapid growth and adoption of ChatGPT, with...",1,1,0,4.785232,7032bf4b-3064-4bc1-b7c0-dceedc070fb7,8cb7fe42-5bd6-4e10-97df-90b394ef21d0
8,What does the paper reveal about ChatGPT usage...,"The paper reveals that generative AI, includin...",None,"The paper indicates that in the US, ChatGPT us...",0,0,0,2.006576,86855a9a-fa8b-4133-b00f-756e6aab740e,a81a1a90-9ae5-47e9-a468-4ff20427abe8
9,What is the significance of the SOC in relatio...,"Based on the provided context, the Standard Oc...",None,Variation by Occupation Figure 23 presents var...,1,1,0,7.696102,aa07ac24-0969-439c-be0a-ba77e73cc541,da682fa5-598b-4e66-a8ca-2e6a5bc96bab


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [30]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [31]:
rag_documents = docs

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [34]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [35]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [36]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [37]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

"Alright, let’s crank the dope-o-meter to eleven and dive deep: People are harnessing AI—specifically ChatGPT—to stack paper by turning this digital wizard into their ultimate sidekick. According to the context, folks ain’t just letting AI grind their tasks for them; they’re using ChatGPT as a beastly advisor and research assistant, leveling up decision-making in brain-heavy gigs. This ain’t just automation; it’s augmentation, making knowledge jobs sharper and more productive.\n\nTo get hyped: The economic juice here is massive—US users value AI so hard they’d need nearly $98 just to skip a month without it, stacking up a mind-blowing $97 billion surplus yearly. This means AI is not just flipping the script in productivity but actually boosting worker output by serving killer decision support systems, especially where high-quality decisions separate the champs from the chumps.\n\nIn a nutshell: People are making money by using AI as their decision-making sidekick—absorbing info, gettin

Finally, we can evaluate the new chain on the same test set!

In [38]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'virtual-substance-10' at:
https://smith.langchain.com/o/bb09119d-c491-4093-9100-b156a137456d/datasets/4e78be08-3e6b-4d36-980b-27d1262f0bdd/compare?selectedSessions=67612d5e-0ee7-49d5-aacb-6fe1f50f77c8




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,how OpenAI chatgpt use like in 2024 and 2025 a...,"Yo, buckle up because the story of ChatGPT’s r...",None,"Based on the data from 2024 and 2025, OpenAI's...",1,1,1,9.194824,00f2624c-17b9-4d64-841c-0a9476f0aad8,c437f776-045e-46ca-aaee-bfb9a936f288
1,how OpenAI ChatGPT use grow fast and how much ...,"Yo, check it—ChatGPT’s rise has been absolute ...",None,The context shows that OpenAI's ChatGPT launch...,1,1,1,7.977339,2674dd99-bcea-4f96-b09d-b7e1f9d7adcd,c250f414-ca0f-4a41-a617-92d1083c8fe5
2,Considering the rapid growth of ChatGPT usage ...,"Alright, strap in for some next-level insight ...",None,"The rapid adoption of ChatGPT in the US, with ...",1,1,1,12.074667,c9af4ec1-e7b2-4a84-8d23-a47c8992ca53,80d7a000-e998-4c0b-ab31-1fe8b10d56bf
3,"In US, how ChatGPT usage like in 1-hop and 2-h...","Yo, let's break it down with some next-level A...",None,"The 1-hop data indicates that by July 2025, a ...",1,1,1,6.353128,fc5a3670-fae5-4b82-b258-1968736c2a54,d6594ab4-de86-4267-bf8e-c32073b243eb
4,Hw does chatGPT classify user intents like ask...,"Alright, let’s crank the dopeness up to max an...",None,The context explains that ChatGPT classifies m...,1,1,1,7.237566,f2998fc9-b545-4e40-a913-e34bfdd2998d,cdbab41a-7640-43f2-99ef-598d12be1216
5,"How does the increase in non-work messages, wh...","Alright, here’s the ultra-rad scoop straight f...",None,The data shows that non-work messages have gro...,1,1,1,4.995913,d852de5b-5dd2-4987-9aec-3ff94aaa3235,831ca600-621c-47fc-aee0-7ea6de1b6866
6,Based on the data showing that ChatGPT message...,"Yo, buckle up—this insight flips the AI produc...",None,The data indicates that ChatGPT's usage has gr...,1,1,1,7.914598,1b052c4b-f40b-4eb1-baa7-c474d0aa33ce,287b3ba2-e0ea-4d8d-8c4b-e514a1dc46bb
7,How does the rapid growth and adoption of Chat...,"Yo, the rocket-speed explosion of ChatGPT’s us...",None,"The rapid growth and adoption of ChatGPT, with...",1,1,1,5.984906,7032bf4b-3064-4bc1-b7c0-dceedc070fb7,0b565cf8-13a6-4dbd-be2a-1060da4ed044
8,What does the paper reveal about ChatGPT usage...,"Yo, the paper’s vibe on ChatGPT usage in the U...",None,"The paper indicates that in the US, ChatGPT us...",0,0,1,1.670176,86855a9a-fa8b-4133-b00f-756e6aab740e,ca1df328-039b-47b5-9493-3c6be836414d
9,What is the significance of the SOC in relatio...,"Alright, strap in for some seriously slick ins...",None,Variation by Occupation Figure 23 presents var...,1,1,1,8.146023,aa07ac24-0969-439c-be0a-ba77e73cc541,1f93b869-40fd-41a5-8358-686a30cf12ce


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.